## Homework 9: Text Classification with Fine-Tuned BERT

### Due: Midnight on November 5th (with 2-hour grace period) — Worth 85 points

In this final homework, we’ll explore **fine-tuning a pre-trained Transformer model (BERT)** for text classification using the **IMDB Movie Review** dataset. You’ll begin with a working baseline notebook and then conduct a series of controlled experiments to understand how data size, context length, and model architecture affect performance.

You’ll complete three problems:

* **Problem 1:** Evaluate how **sequence length** and **learning rate** jointly influence validation loss and generalization.
* **Problem 2:** Measure how **training data size** affects both model performance and total training time.
* **Problem 3:** Compare **two additional models** from the BERT family to analyze the trade-offs between model size and accuracy on this dataset.

In each problem, you’ll report your key metrics, summarize what you observed, and reflect on what you learned.

> **Note:** This homework was developed and tested on **Google Colab**, due to version conflicts when running locally. It is **strongly recommended** that you complete your work on Colab as well.

There are 6 problems, each worth 14 points, and you get one point free if you complete the entire homework.


In [1]:
# Install once per new Colab runtime
%pip -q install -U keras keras-hub tensorflow tensorflow-text datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 56.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tens

In [2]:

import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import time
import random
import numpy as np
import keras
import keras_hub as kh
import evaluate
from datasets import load_dataset, Dataset, Features, Value, ClassLabel

from keras import mixed_precision                    # generally faster
mixed_precision.set_global_policy("mixed_float16")

### Here is where you can set global hyperparameters for this homework

In [3]:
# ---------------- Config ----------------
SEED        = 42
MAX_LEN     = 128
EPOCHS      = 3
BATCH       = 32
EVAL_BATCH  = 64
SUBSET_FRAC = 0.25   # <-- 0.25 to train and test on 25% of whole dataset during development;  set to 1.0 for full dataset

keras.utils.set_random_seed(SEED)

### Load and Preprocess the IMDB Movie Review Dataset

In [4]:
# ---- Load IMDb (raw), join train+test ----
imdb   = load_dataset("imdb")
texts  = list(imdb["train"]["text"]) + list(imdb["test"]["text"])
labels = np.array(list(imdb["train"]["label"]) + list(imdb["test"]["label"]), dtype="int32")

# ---- Build DS with explicit features (label=ClassLabel) ----
features = Features({"text": Value("string"),
                     "label": ClassLabel(num_classes=2, names=["NEG","POS"])})
all_ds = Dataset.from_dict({"text": texts, "label": labels.tolist()}, features=features)

# ---- Optional: take a stratified subset of the FULL dataset ----
if 0.0 < SUBSET_FRAC < 1.0:
    sub = all_ds.train_test_split(train_size=SUBSET_FRAC, seed=SEED, stratify_by_column="label")
    ds_pool = sub["train"]
else:
    ds_pool = all_ds

# ---- Stratified 80/10/10 split on the (possibly smaller) pool ----
# First: 80/20 train+val pool / test
splits = ds_pool.train_test_split(test_size=0.20, seed=SEED, stratify_by_column="label")
train_val_pool, test_ds = splits["train"], splits["test"]
# Then: carve 10% of full (i.e., 0.125 of the 80% pool) as validation
splits2 = train_val_pool.train_test_split(test_size=0.125, seed=SEED, stratify_by_column="label")
train_ds, val_ds = splits2["train"], splits2["test"]

# ---- Numpy arrays for Keras fit/predict ----
X_tr = np.array(train_ds["text"], dtype=object); y_tr = np.array(train_ds["label"], dtype="int32")
X_va = np.array(val_ds["text"],   dtype=object); y_va = np.array(val_ds["label"],   dtype="int32")
X_te = np.array(test_ds["text"],  dtype=object); y_te = np.array(test_ds["label"],  dtype="int32")

# ---- Quick summary ----
def _counts(ds):
    arr = np.array(ds["label"], dtype=int)
    return len(arr), np.bincount(arr, minlength=2).tolist()
print(f"Pool after SUBSET_FRAC={SUBSET_FRAC}: {len(ds_pool)} (of {len(all_ds)})")
print("Train:", _counts(train_ds), " Val:", _counts(val_ds), " Test:", _counts(test_ds))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Pool after SUBSET_FRAC=0.25: 12500 (of 50000)
Train: (8750, [4375, 4375])  Val: (1250, [625, 625])  Test: (2500, [1250, 1250])


### Build and train a baseline Distil-Bert Text Classifier

In [5]:
# ---- Keras Hub preprocessor + classifier ----
preproc = kh.models.DistilBertTextClassifierPreprocessor.from_preset(
    "distil_bert_base_en_uncased", sequence_length=MAX_LEN
)
model = kh.models.DistilBertTextClassifier.from_preset(
    "distil_bert_base_en_uncased", num_classes=2, preprocessor=preproc
)

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

start = time.time()

# ---- Train with early stopping (restore best val weights) ----
cb = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)]
history = model.fit(
    X_tr, y_tr,
    validation_data=(X_va, y_va),
    epochs=EPOCHS,
    batch_size=BATCH,
    callbacks=cb,
    verbose=1,
)

# ---- Evaluate (accuracy + F1 via `evaluate`) ----
logits = model.predict(X_te, batch_size=EVAL_BATCH, verbose=0)
y_pred = logits.argmax(axis=-1)

acc_metric = evaluate.load("accuracy")
f1_metric  = evaluate.load("f1")
acc = acc_metric.compute(predictions=y_pred, references=y_te)["accuracy"]
f1  = f1_metric.compute(predictions=y_pred, references=y_te)["f1"]

# Tiny confusion matrix helper (no sklearn needed)
def confusion_matrix_np(y_true, y_pred, num_classes=2):
    cm = np.zeros((num_classes, num_classes), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm

print(f"\nValidation acc (best epoch): {history.history['val_acc'][np.argmin(history.history['val_loss'])]:.3f}")
print(f"\nTest accuracy: {acc:.3f}   Test F1: {f1:.3f}")
print("\nConfusion matrix:\n", confusion_matrix_np(y_te, y_pred))

end = time.time() - start
print("\nElapsed time:", time.strftime("%H:%M:%S", time.gmtime(end)))

Epoch 1/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 124s 250ms/step - acc: 0.7827 - loss: 0.4529 - val_acc: 0.8376 - val_loss: 0.3447
Epoch 2/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - acc: 0.8786 - loss: 0.2895 - val_acc: 0.8584 - val_loss: 0.3399
Epoch 3/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - acc: 0.9159 - loss: 0.2205 - val_acc: 0.8600 - val_loss: 0.3558



Validation acc (best epoch): 0.858

Test accuracy: 0.854   Test F1: 0.851

Confusion matrix:
 [[1095  155]
 [ 209 1041]]

Elapsed time: 00:02:51


# Problem 1 — Mini sweep: context length × learning rate (6 runs)

In this problem we'll see how much **context length** (`MAX_LEN`) helps, and how sensitive fine-tuning is to **learning rate**—without running a huge grid.

## Setup (keep these fixed)

* `SUBSET_FRAC = 0.25`               # use only this percentage of the whole dataset
* `EPOCHS = 3`
* `BATCH = 32` (but see note for 256 below)
* **EarlyStopping** with `restore_best_weights=True`
* Same random `SEED` for all runs
* Same data split for all runs (don’t reshuffle between runs)

### Run these 6 configurations

**For each** `MAX_LEN ∈ {128, 256, 512}`, try **two** learning rates:

* **MAX_LEN = 128**

  * `(LR = 2e-5, BATCH = 32)` – healthy default for shorter contexts.
  * `(LR = 1e-5, BATCH = 32)` – conservative LR; often a touch stabler.

* **MAX_LEN = 256**

  * `(LR = 1e-5, BATCH = 16)` – longer context → lower batch.
  * `(LR = 7.5e-6, BATCH = 16)` – even steadier if loss is noisy.

* **MAX_LEN = 512**  *(heavier quadratic attention cost)*

  * `(LR = 7.5e-6, BATCH = 8)` – safe starting point.
  * `(LR = 5e-6, BATCH = 8)` – extra caution for stability.

**If you hit an Out Of Memory error:**

* At **256** with `BATCH = 16`, drop to `BATCH = 8`.
* At **512** with `BATCH = 8`, drop to `BATCH = 4`.


Then answer the graded questions.


In [24]:
# Your code here; add as many cells as you need
max_len_dict = {
    '128':[[128, 2e-5, 32],[128, 1e-5, 32]],
    '256':[[256, 1e-5, 16],[256, 7.5e-6, 16]],
    '512':[[512, 7.5e-6, 8],[512, 5e-6, 8]]
}

for i in max_len_dict.keys():
  for j in max_len_dict[i]:
    print(f"Running Bert model with Max Sequance length: {j[0]}, Learning Rate: {j[1]}, and Batch Size {j[2]} ...")

    preproc_1 = kh.models.DistilBertTextClassifierPreprocessor.from_preset(
        "distil_bert_base_en_uncased", sequence_length=j[0]
    )
    model = kh.models.DistilBertTextClassifier.from_preset(
        "distil_bert_base_en_uncased", num_classes=2, preprocessor=preproc
    )

    model.compile(
        optimizer=keras.optimizers.Adam(j[1]),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )

    start = time.time()

    # ---- Train with early stopping (restore best val weights) ----
    cb = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)]
    history = model.fit(
        X_tr, y_tr,
        validation_data=(X_va, y_va),
        epochs=EPOCHS,
        batch_size=j[2],
        callbacks=cb,
        verbose=1,
    )

    # ---- Evaluate (accuracy + F1 via `evaluate`) ----
    logits = model.predict(X_te, batch_size=EVAL_BATCH, verbose=0)
    y_pred = logits.argmax(axis=-1)

    acc_metric = evaluate.load("accuracy")
    f1_metric  = evaluate.load("f1")
    acc = acc_metric.compute(predictions=y_pred, references=y_te)["accuracy"]
    f1  = f1_metric.compute(predictions=y_pred, references=y_te)["f1"]

    # ---- Print the evaluations ----
    print(f"Running Bert model with Max Sequance length: {j[0]}, Learning Rate: {j[1]}, and Batch Size {j[2]} ...")
    print(f"\nValidation acc (best epoch): {history.history['val_acc'][np.argmin(history.history['val_loss'])]:.3f}")
    print(f"\nTest accuracy: {acc:.3f}   Test F1: {f1:.3f}")
    print("\nConfusion matrix:\n", confusion_matrix_np(y_te, y_pred))

    end = time.time() - start
    print("\nElapsed time:", time.strftime("%H:%M:%S", time.gmtime(end)))


Running Bert model with Max Sequance length: 128, Learning Rate: 2e-05, and Batch Size 32 ...
Epoch 1/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 89s 162ms/step - acc: 0.8054 - loss: 0.4123 - val_acc: 0.8512 - val_loss: 0.3367
Epoch 2/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - acc: 0.9013 - loss: 0.2497 - val_acc: 0.8440 - val_loss: 0.3884
Epoch 3/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - acc: 0.9358 - loss: 0.1734 - val_acc: 0.8520 - val_loss: 0.3729
Running Bert model with Max Sequance length: 128, Learning Rate: 2e-05, and Batch Size 32 ...

Validation acc (best epoch): 0.851

Test accuracy: 0.851   Test F1: 0.855

Confusion matrix:
 [[1035  215]
 [ 157 1093]]

Elapsed time: 00:02:00
Running Bert model with Max Sequance length: 128, Learning Rate: 1e-05, and Batch Size 32 ...
Epoch 1/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 89s 159ms/step - acc: 0.7845 - loss: 0.4510 - val_acc: 0.8536 - val_loss: 0.3467
Epoch 2/3
274/274 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - acc: 0.8810 - loss: 0.2903 - val_acc: 0

### Graded Questions

In [25]:
# Set a1a to the validation accuracy at min validation loss for your best configuration found in this problem

a1a = 0.868          # Replace 0.0 with your answer

In [26]:
# Graded Answer
# DO NOT change this cell in any way

print(f'a1a = {a1a:.4f}')

a1a = 0.8680


#### Question a1b:

* Does **more context** (128 → 256 → 512) consistently help?
* How much effect did the learning rate have on the validation accuracy?


#### Your Answer Here: In general the higher context models performed better but it wasn't by a large margin. The learning rates impacted the models but not in a consistent manner, the combination of sequance length and learning rate may have hit an optimal low for one of the variants but performed moderately with a different combination.

## Problem 2 — How much data is enough?

In this problem, you’ll investigate how model performance scales with dataset size.

**Setup.**
Use the best `MAX_LEN` and `LR` values you found in **Problem 1**.

**What to do:**

1. For each value of `SUBSET_FRAC ∈ {0.25, 0.50, 0.75, 1.00}`, train your model once and observe the displayed performance metrics.
2. Answer the discussion question below.




In [ ]:
# Your code here; add as many cells as you need


### Graded Questions

In [ ]:
# Set a2a to the validation accuracy at min validation loss for your best configuration found in this problem
# (Yes, it is probably at 1.0!)

a2a = 0.0             # Replace 0.0 with your answer

In [ ]:
# Graded Answer
# DO NOT change this cell in any way

print(f'a2a = {a2a:.4f}')

#### Question a2b:

Summarize what you observed as dataset size increased. Given that validation metrics are typically reliable to only about two decimal places, do the performance gains justify using the entire dataset? What trade-offs between accuracy and computation time did you notice?

#### Your Answer Here:

# Problem 3 — Model swap: speed vs. accuracy (why: capacity matters)

In this problem we will compare encoder-only backbones of different sizes.

**Setup.** Keep the best `MAX_LEN`, `LR`, and `SUBSET_FRAC` from Problems 1–2. Only change the model/preset:

* **DistilBERT** (current baseline)
* **BERT-base** (larger/usually stronger)

**How to switch (two lines each).**

* DistilBERT:

  ```python
  preproc = kh.models.DistilBertTextClassifierPreprocessor.from_preset("distil_bert_base_en_uncased", sequence_length=MAX_LEN)
  model  = kh.models.DistilBertTextClassifier.from_preset("distil_bert_base_en_uncased", num_classes=2, preprocessor=preproc)
  ```

* BERT-base:

  ```python
  preproc = kh.models.BertTextClassifierPreprocessor.from_preset("bert_base_en_uncased", sequence_length=MAX_LEN)
  model  = kh.models.BertTextClassifier.from_preset("bert_base_en_uncased", num_classes=2, preprocessor=preproc)
  ```

**What to do.**

1. Train/evaluate each model once with identical settings.
2. Observe the performance metrics for each.
3. Answer the graded questions.



In [ ]:
# Your code here; add as many cells as you wish



### Graded Questions

In [ ]:
# Set a1a to the validation accuracy at min validation loss for your best model found in this problem

a3a = 0.0             # Replace 0.0 with your answer

In [ ]:
# Graded Answer
# DO NOT change this cell in any way

print(f'a3a = {a3a:.4f}')

#### Question a3b:

**Answer briefly.**

* Which model gives the best **accuracy/F1**?
* Which is **fastest** per epoch?
* Given limited development time or compute resources, which model is the best **overall choice** and why?

#### Your Answer Here: